**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!mkdir "/mydrive/yolov3"

In [ ]:
%cd gdrive/MyDrive/yolov3
!ls

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights


In [ ]:
!echo "gengel" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Create train and text files with images full address paths

In [ ]:
%cd data/obj/images

/content/gdrive/My Drive/yolov3/darknet/data/obj/images


In [ ]:
import glob
import os

images_list = glob.glob("*.jpg")
print(images_list)

['gengel16.jpg', 'gengel15.jpg', 'gengel29.jpg', 'gengel28.jpg', 'gengel14.jpg', 'gengel38.jpg', 'gengel10.jpg', 'gengel8.jpg', 'gengel9.jpg', 'gengel11.jpg', 'gengel39.jpg', 'gengel13.jpg', 'gengel12.jpg', 'gengel75.jpg', 'gengel61.jpg', 'gengel49.jpg', 'gengel48.jpg', 'gengel60.jpg', 'gengel74.jpg', 'gengel62.jpg', 'gengel76.jpg', 'gengel77.jpg', 'gengel63.jpg', 'gengel67.jpg', 'gengel73.jpg', 'gengel72.jpg', 'gengel66.jpg', 'gengel58.jpg', 'gengel70.jpg', 'gengel64.jpg', 'gengel65.jpg', 'gengel71.jpg', 'gengel59.jpg', 'gengel54.jpg', 'gengel40.jpg', 'gengel68.jpg', 'gengel69.jpg', 'gengel41.jpg', 'gengel55.jpg', 'gengel80.jpg', 'gengel43.jpg', 'gengel56.jpg', 'gengel42.jpg', 'gengel46.jpg', 'gengel52.jpg', 'gengel53.jpg', 'gengel47.jpg', 'gengel79.jpg', 'gengel51.jpg', 'gengel45.jpg', 'gengel44.jpg', 'gengel50.jpg', 'gengel78.jpg', 'gengel37.jpg', 'gengel23.jpg', 'gengel7.jpg', 'gengel6.jpg', 'gengel22.jpg', 'gengel36.jpg', 'gengel20.jpg', 'gengel34.jpg', 'gengel4.jpg', 'gengel5.jpg

In [ ]:
images_adress_list = ["/content/gdrive/My Drive/yolov3/darknet/data/obj/images/"+ i for i in images_list]
print(images_adress_list)

['/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel16.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel15.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel29.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel28.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel14.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel38.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel10.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel8.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel9.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel11.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel39.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel13.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel12.jpg', '/content/gdrive/My Drive/yolov3/darknet/data/obj/images/gengel75

In [ ]:
%cd ..

/content/gdrive/My Drive/yolov3/darknet


In [ ]:
train_file = open("data/train.txt", "w")
train_file.write("\n".join(images_adress_list[:70]))

test_file = open("data/test.txt", "w")
test_file.write("\n".join(images_adress_list[70:]))

train_file.close()
test_file.close()

**Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show